In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# source activate tensorflow_p36
# pip install sklearn losswise

In [3]:
import pickle
#from cv2 import imread, resize
from PIL import Image
from pathlib import Path
import random
import calendar
import time
from random import uniform
from sklearn.metrics import f1_score

import numpy as np
from keras.applications.xception import Xception
from keras.layers import Input, Conv2D, merge, Dense, Flatten, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.models import Model, Sequential
from keras.optimizers import SGD, Adam
from keras import backend as K

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model

from keras.callbacks import ModelCheckpoint, TensorBoard

Using TensorFlow backend.


In [4]:
from losswise.libs import LosswiseKerasCallback
import losswise
losswise.set_api_key('JWN8A6X96')

In [5]:
K.image_data_format() == 'channels_last'

True

In [6]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [7]:
save_path = '/home/ubuntu/Dimitris/'
data_path = '/home/ubuntu/data/'
images_path = '/home/ubuntu/temp/images/jpeg/'
csv_path = data_path + 'ConceptDetectionTraining2018-Concepts.csv'

results_path = save_path + r'results/'
val_size = 4096

### create and save lists and dicts

### load lists and dicts

In [8]:
csv_name_labels_dict = load_obj(save_path + 'csv_name_labels_dict')
names_list = load_obj(save_path + 'names_list')
labels_list = load_obj(save_path + 'labels_list')

train_names_list = load_obj(save_path + 'train_names_list')
train_name_labels_dict = load_obj(save_path + 'train_name_labels_dict')

val_names_list = load_obj(save_path + 'val_names_list')
val_name_labels_dict = load_obj(save_path + 'val_name_labels_dict')

labels_num = len(labels_list)

In [9]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.

    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.

    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.

    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)

### create generators

In [10]:
def images_generator(this_name_labels_dict, this_names_list,
                    batch_size, img_size=(224, 224, 3), suffix='.jpg'):
    while True:
        imgs = np.zeros((batch_size, *img_size))
        labels = np.zeros((batch_size, labels_num))
        i = 0
        while i < batch_size:
            name = random.choice(this_names_list)
            filename = images_path + name + suffix
            img = load_img(filename, target_size=img_size[:2])
            img = img_to_array(img)
            img = img.reshape((1, img.shape[0], img.shape[1], img.shape[2]))
            img = preprocess_input(img, mode='tf')
            
            # flip horizontaly
            if uniform(0, 1) > 0.5:
                img = np.fliplr(img)
            
            # add noise
            if uniform(0, 1) > 0.5:
                noise = np.random.normal(0, 0.05, img.shape)
                img = img + noise
                img = np.clip(img, -1, 1)
            
            imgs[i] = img
            for label in this_name_labels_dict[name]:
                ind = labels_list.index(label)
                labels[i, ind] = 1
            i += 1
        yield imgs, labels

In [11]:
batch_size = 32
val_batch_size = 64

input_shape = (224, 224, 3)
suffix = '.jpg'

train_gen = images_generator(train_name_labels_dict, train_names_list,
                             batch_size, img_size=input_shape, suffix=suffix)

val_gen = images_generator(val_name_labels_dict, val_names_list,
                             val_batch_size, img_size=input_shape, suffix=suffix)

In [12]:
suffix

'.jpg'

## Make val arrays

In [13]:
# x_train = load_obj(save_path + 'x_train')
# y_train = load_obj(save_path + 'y_train')

x_test = load_obj(save_path + 'x_test')
y_test = load_obj(save_path + 'y_test')

### create Xception model

### compilation and training parameters

In [14]:
first_phase_epochs = 10
second_phase_epochs = 10
saves_per_epoch = 10
small_epochs = 50

imgs_per_rep = int(len(train_names_list) / saves_per_epoch)
imgs_per_small_epoch = int(imgs_per_rep / small_epochs)
steps_per_small_epoch = int(imgs_per_small_epoch / batch_size)

steps_per_small_epoch = int(imgs_per_small_epoch / batch_size)

first_phase_train_reps = first_phase_epochs * saves_per_epoch
second_phase_train_reps = second_phase_epochs * saves_per_epoch

# val_imgs_per_rep = int(val_size / saves_per_epoch)
# val_imgs_per_small_epoch = int(val_imgs_per_rep / small_epochs)
val_steps_per_small_epoch = int(val_size / batch_size)

big_train_steps_per_epoch = int(len(train_names_list) / batch_size)
big_val_steps_per_epoch = int(len(val_names_list) / batch_size)

# f1 score

### first compile

### train last layers of model (1st step)

In [15]:
checkpointer = ModelCheckpoint(filepath=results_path + 'weights.{epoch:02d}-{val_fmeasure:.2f}.hdf5', verbose=1)
tensorBoard = TensorBoard(log_dir= results_path + r'tensorBoard/logs')

### second compile

In [16]:
import keras

In [17]:
#xcpetion_model = load_model(results_path + 'first_phase' + '_xcpetion_model.h5', custom_objects={'fmeasure': fmeasure})
#trainable_layers_ratio = 1 / 2.0
#trainable_layers_index = int(len(xcpetion_model.layers) * (1 - trainable_layers_ratio))
#for layer in xcpetion_model.layers[:trainable_layers_index]:
#    layer.trainable = False
#for layer in xcpetion_model.layers[trainable_layers_index:]:
#    layer.trainable = True
# for layer in xcpetion_model.layers[:]:
#     layer.trainable = True

xcpetion_model = load_model('/home/ubuntu/Dimitris/results/' + 'second_v2_phase_xcpetion_model.h5', custom_objects={'fmeasure': fmeasure})

# predictions = xcpetion_model.layers[-1]
# dropout = Dropout(0.3, name='fc_1024_dropout', seed=0)
# fc = xcpetion_model.layers[-2]

# x = dropout(fc.output)
# predictors = predictions(x)
# new_xcpetion_model = Model(inputs=xcpetion_model.input, outputs=predictors)

# new_xcpetion_model.compile(optimizer=Adam(lr=0.0005), loss='categorical_crossentropy', metrics=['acc'])





In [18]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
xcpetion_model.compile(optimizer=Adam(lr=0.0005), loss='binary_crossentropy', metrics=[fmeasure])
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

### fine tune half layers of model (2nd step)

In [19]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []
        self.fmeasures = []
        self.val_fmeasures = []
    def on_batch_end(self, batch, logs={}):
        with open('/home/ubuntu/Dimitris/results/' + 'logs_v3.csv', "a") as myfile:
            myfile.write(','.join([
                 "" if not logs.get('loss') else str(logs.get('loss')),
                 "" if not logs.get('val_loss') else str(logs.get('val_loss')),
                 "" if not logs.get('fmeasure') else str(logs.get('fmeasure')),
                 "" if not logs.get('val_fmeasure') else str(logs.get('val_fmeasure'))
            ]) + '\n') 
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.fmeasures.append(logs.get('fmeasure'))
        self.val_fmeasures.append(logs.get('val_fmeasure'))

In [20]:
# # train the model on the new data for a few epochs
# for i in range(second_phase_train_reps):
#     history = xcpetion_model.fit_generator(train_gen,
#                                  steps_per_epoch=steps_per_small_epoch,
#                                  epochs=small_epochs, verbose=2,
#                                  validation_data=val_gen, validation_steps=val_steps_per_small_epoch)
#     print(i)
#     if i % saves_per_epoch == 0:
#         print('{} epoch completed'.format(int(i / saves_per_epoch)))
    
#     ts = calendar.timegm(time.gmtime())
#     xcpetion_model.save(save_path + str(ts) + '_xcpetion_model.h5')
#     save_obj(history.history, save_path + str(ts) + '_xcpetion_history.h5')
#checkpointer = ModelCheckpoint(filepath=results_path + 'second_v3_phase_weights.{epoch:02d}-{val_fmeasure:.2f}.hdf5', verbose=1)
history = LossHistory()

history = xcpetion_model.fit_generator(train_gen, steps_per_epoch=big_train_steps_per_epoch,
                                       epochs=second_phase_epochs, verbose=1, callbacks=[tensorBoard, history, LosswiseKerasCallback(tag='keras xcpetion model for ImageCLEF')],
                                       validation_data=(x_test, y_test), workers=1)
# ts = calendar.timegm(time.gmtime())
xcpetion_model.save(results_path + 'second_v3_phase' + '_xcpetion_model.h5')
save_obj(history.history, results_path + 'second_v3_phase' + '_xcpetion_history')

Epoch 1/10
6867/6867 [==============================] - 4529s 660ms/step - loss: 9.9476e-04 - fmeasure: 0.2893 - val_loss: 0.0013 - val_fmeasure: 0.2243
Epoch 2/10
6867/6867 [==============================] - 4516s 658ms/step - loss: 9.6233e-04 - fmeasure: 0.3058 - val_loss: 0.0014 - val_fmeasure: 0.2232
Epoch 3/10
6867/6867 [==============================] - 4510s 657ms/step - loss: 9.2809e-04 - fmeasure: 0.3241 - val_loss: 0.0014 - val_fmeasure: 0.2324
Epoch 4/10
6867/6867 [==============================] - 4518s 658ms/step - loss: 8.9184e-04 - fmeasure: 0.3450 - val_loss: 0.0014 - val_fmeasure: 0.2301
Epoch 5/10
6867/6867 [==============================] - 4536s 661ms/step - loss: 8.5824e-04 - fmeasure: 0.3675 - val_loss: 0.0015 - val_fmeasure: 0.2338
Epoch 6/10
6867/6867 [==============================] - 4534s 660ms/step - loss: 8.2270e-04 - fmeasure: 0.3887 - val_loss: 0.0015 - val_fmeasure: 0.2300
Epoch 7/10
6867/6867 [==============================] - 4535s 660ms/step - loss: 7